# State Management Examples

This notebook demonstrates the `State` class, a persistent key-value store designed for pipeline state management. It maintains intermediate results and configuration across pipeline steps.

## Features Demonstrated

- **Basic Operations**: Dictionary-like interface for storing and retrieving values
- **Readonly Views**: Immutable state views to prevent accidental modifications
- **Scoped Views**: Namespace-based access using dot notation (e.g., `train.lr`)
- **Type Safety**: Runtime type validation with `check_type()` and `get_typed()`
- **Error Handling**: Proper exception handling for missing keys and type mismatches

## Setup

In [1]:
import sys
import os

# Add the project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from src.idspy.core.state import State

s = State({"seed": 42, "train.lr": 1e-3})
s

State(size=2, data={'seed': 42, 'train.lr': 0.001})

## Basic Operations

Dictionary-like interface for storing and retrieving values.

In [3]:
s["model"] = "resnet18"
s["model"]

'resnet18'

## Readonly Views

Create immutable views to prevent accidental modifications.

In [4]:
ro = s.readonly()
print(f"reading: {ro["seed"]}")

try:
    ro["seed"] = 123
except TypeError as e:
    print("readonly:", e)

reading: 42
readonly: 'mappingproxy' object does not support item assignment


## Scoped Views

Access subsets using namespace prefixes (e.g., `train.*`).

In [5]:
train = s.scope("train")
list(train)

['lr']

### Modifying Through Scopes

Modifications through scoped views update the main state.

In [6]:
train["epochs"] = 10  # adds "train.epochs" to the state
s.to_dict()

{'seed': 42, 'train.lr': 0.001, 'model': 'resnet18', 'train.epochs': 10}

### Deletion Through Scopes

Delete entries from the main state using scoped views.

In [7]:
del train["lr"]  # removes "train.lr" from state
s

State(size=3, data={'seed': 42, 'model': 'resnet18', 'train.epochs': 10})

## Type Safety

Runtime type validation for stored values.

In [8]:
# Setup test data with various types
s["learning_rate"] = 0.001
s["batch_size"] = 32
s["model_name"] = "resnet18"
s["is_training"] = True
s["layers"] = [64, 128, 256]

print("Current state:")
print(s.to_dict())

Current state:
{'seed': 42, 'model': 'resnet18', 'train.epochs': 10, 'learning_rate': 0.001, 'batch_size': 32, 'model_name': 'resnet18', 'is_training': True, 'layers': [64, 128, 256]}


### Type Checking

Check if a key exists and matches expected type without exceptions.

In [9]:
# Check types - returns True/False
print("Type checks:")
print(f"learning_rate is float: {s.check_type('learning_rate', float)}")
print(f"batch_size is int: {s.check_type('batch_size', int)}")
print(f"model_name is str: {s.check_type('model_name', str)}")
print(f"is_training is bool: {s.check_type('is_training', bool)}")
print(f"layers is list: {s.check_type('layers', list)}")

# Check wrong types
print(f"learning_rate is int: {s.check_type('learning_rate', int)}")
print(f"nonexistent_key is str: {s.check_type('nonexistent_key', str)}")

Type checks:
learning_rate is float: True
batch_size is int: True
model_name is str: True
is_training is bool: True
layers is list: True
learning_rate is int: False
nonexistent_key is str: False


### Typed Access

Retrieve values with guaranteed type safety. Raises exceptions for mismatches.

In [10]:
# Successful typed access
lr = s.get_typed("learning_rate", float)
batch_sz = s.get_typed("batch_size", int)
model = s.get_typed("model_name", str)

print("Successful typed access:")
print(f"Learning rate (float): {lr}")
print(f"Batch size (int): {batch_sz}")
print(f"Model name (str): {model}")

Successful typed access:
Learning rate (float): 0.001
Batch size (int): 32
Model name (str): resnet18


In [11]:
# Error handling examples
print("\nError handling:")

# KeyError for missing key
try:
    s.get_typed("missing_key", str)
except KeyError as e:
    print(f"KeyError: {e}")

# TypeError for wrong type
try:
    s.get_typed("learning_rate", int)  # learning_rate is float, not int
except TypeError as e:
    print(f"TypeError: {e}")

# Another type mismatch
try:
    s.get_typed("batch_size", str)  # batch_size is int, not str
except TypeError as e:
    print(f"TypeError: {e}")


Error handling:
KeyError: "Key 'missing_key' not found in state."
TypeError: Expected value of type <class 'int'> for key 'learning_rate', got <class 'float'>.
TypeError: Expected value of type <class 'str'> for key 'batch_size', got <class 'int'>.
